# Obtención de datos desde la API del Banco Mundial

In [1]:
import requests
import pandas as pd

## Extracción de los datos de la variable objetivo: Esperanza de Vida al Nacer

Según David, Rodríguez (2015) los factores que inciden sobre la Esperanza de Vida al Nacer dependen en gran medida de los ingresos per capita del país. Por ello vamos a extraer datos de paises que pertenezcan a cada una de las 4 categorías: Ingresos Bajos, Ingresos Medio - Bajos, Ingresos Medio - Altos e Ingresos Altos. Todo ello según la clasificación del Banco Mundial

Aqui vemos los codigos para cada categoría que sera el argumento para la extraccion de los paises

In [19]:
url = 'http://api.worldbank.org/v2/incomelevel' #Aqui vemos la lista de niveles de Ingreso y sus respectivos códigos
args = {"format":"json", "prefix":"Getdata"}
dic_Income_Level = requests.get(url, params=args)
datos = dic_Income_Level.json()[1]
data = pd.json_normalize(datos)
data[['value', 'id']]

,value,id
0,High income,HIC
1,Not classified,INX
2,Low income,LIC
3,Lower middle income,LMC
4,Low & middle income,LMY
5,Middle income,MIC
6,Upper middle income,UMC


Ahora vamos a tomar una muestra de 10 paises de cada categoría

In [24]:
url = 'http://api.worldbank.org/v2/country' #Aqui vemos la lista de niveles de Ingreso y sus respectivos códigos
args = {"format":"json", "prefix":"Getdata", "incomelevel":"HIC"}
dic_Income_Level = requests.get(url, params=args)
datos = dic_Income_Level.json()[1]
data = pd.json_normalize(datos)
data[['name', 'id', 'region.value', 'incomeLevel.value']].head(10)

,name,id,region.value,incomeLevel.value
0,Aruba,ABW,Latin America & Caribbean,High income
1,Andorra,AND,Europe & Central Asia,High income
2,United Arab Emirates,ARE,Middle East & North Africa,High income
3,Antigua and Barbuda,ATG,Latin America & Caribbean,High income
4,Australia,AUS,East Asia & Pacific,High income
5,Austria,AUT,Europe & Central Asia,High income
6,Belgium,BEL,Europe & Central Asia,High income
7,Bahrain,BHR,Middle East & North Africa,High income
8,"Bahamas, The",BHS,Latin America & Caribbean,High income
9,Bermuda,BMU,North America,High income


Selección de un solo país bajo estudio. Si se eligen todos los países la API tarda un poco más de 30 segundos en responder una vez terminado el ciclo for. Y esa carga inicial podría molestar al usuario final.

In [14]:
paises=pd.read_csv('../datasets/paises.csv')

seleccion = input("Ingrese un nombre de país: ")
try:
    pais = paises.loc[paises['country.value'].str.contains(seleccion, case=False),'countryiso3code'].values[0]
except:
    print("No se encontró el país, intente de nuevo")

In [15]:
# Establecemos los parámetros que queremos acceder.
indicador = 'SP.DYN.LE00.IN'
#pais = 'all' # Está comentado porque lo recibe del bloque anterior
iyear = '1980'
fyear = '2022'

# Página de la api y path al recurso solicitado
api_url = ' http://api.worldbank.org/v2/es'
path = f'/country/{pais}/indicator/{indicador}'

url = api_url + path

# Creamos el diccionario con los parametros 
# para el método get
def consultar(pagina = 1):
    args= {
        "date":iyear+":"+fyear,
        'page':pagina,
        "per_page":1000,
        "format":"json",
        "prefix":"Getdata",
    }
    return requests.get(url,params=args).json()

dict_data = consultar()

In [16]:
# La primera parte de la respuesta nos indica en 
# cuantas páginas de encuentra la información
paginas=dict_data[0]["pages"]
paginas # Retornamos el número de páginas

1

In [ ]:
# La segunda parte nos retorna una lista de diccionarios 
# con la información que queríamos
datos=dict_data[1]
datos

In [18]:
# Agregamos los valores de las otras páginas, si existen a 
# nuestra lista de diccionarios
for pagina in range(2,paginas+1):
    datos.extend(consultar(pagina)[1])

In [19]:
# Verifico que lo obtenido sea una lista de diccionarios
print(type(datos))
print(type(datos[1]))

<class 'list'>
<class 'dict'>


In [20]:
# Creo el DataFrame con todos los datos
data = pd.json_normalize(datos)

In [ ]:
# Visualizo información relevante unicamente
data[['date','value','country.value']]

In [10]:
# Creo un dataset para trabajar offline
# data.to_csv('../datasets/esperanza_de_vida.csv')

# EDA

Esto se hace en la primera ejecución del código, cuando se corría con todos los países

In [11]:
paises = data[['countryiso3code','country.value']].groupby('countryiso3code').first()

# Regreso el index creado por agrupar a una columna
paises.reset_index(inplace=True)

# Reemplazo todos los valores faltantes por NaN
paises.replace("", float("NaN"), inplace=True)

paises.dropna(inplace=True)

# Creo el dataset de países que me va a servir de selección
# paises.to_csv('../datasets/paises.csv')

Verifico los datos faltantes en la respuesta de la API en español. Son dós nombres de países o grupo de países, con el código AFW y AFE, para Africa ocidental y central y Africa del este y del sur, respectivamente. En resumen: Regiones de Africa. No sé si lo vamos a incluir

In [ ]:
data[data.countryiso3code.str.contains('AFE')]

## Búsqueda de indicadores

In [13]:
url='http://api.worldbank.org/v2/es/indicator'
def consultar_indicadores(pagina=1):
    argumentos= {
        'page':pagina,
        'per_page':10000,
        'format':'json'
    }
    return requests.get(url,params=argumentos).json()

indicadores = consultar_indicadores()

# la primera parte de la respuesta 
# contiene información sobre las páginas
paginas = indicadores[0]["pages"]

# Reasigno la variable únicamente a los datos de mi interés 
# es una lista de diccionarios
indicadores = indicadores[1]

# recorro todás las páginas que tiene la respuesta 
# y lo voy anexando a la lista principal
for pagina in range(2, paginas+1):
    indicadores.extend(consultar_indicadores(pagina)[1])
indicators = pd.json_normalize(indicadores)

# Dev data para trabajo offline
# indicators.to_csv('../datasets/indicadores.csv')

interes = input("Indicador de interés: ")
indicators.loc[indicators.name.str.contains(interes,case=False),
     ['id','name','sourceNote']]#.to_csv('temp.csv')

,id,name,sourceNote
179,3.0.Gini,Coeficiente de Gini,El coeficiente de Gini es la medida de desigua...
180,3.0.Gini_nozero,Coeficiente de Gini (Ingreso diferente de cero),El coeficiente de Gini es la medida de desigua...
192,3.1.Gini,"Gini, Rural",El coeficiente de Gini es la medida de desigua...
205,3.2.Gini,"Gini, Urbano",El coeficiente de Gini es la medida de desigua...
17088,SI.POV.GINI,Índice de Gini,El índice de Gini mide hasta qué punto la dist...
